In [46]:
import pandas as pd

In [47]:
# df0 = pd.read_csv("data.csv")
# df1 = pd.read_csv("data1.csv")
# df2 = pd.read_csv("data2.csv")
# df3 = pd.read_csv("data3.csv")
# df4 = pd.read_csv("data4.csv")
# df5 = pd.read_csv("data5.csv")
# df6 = pd.read_csv("data6.csv")

In [48]:
df0 = pd.read_csv("data0.csv")
df2 = pd.read_csv("data2.csv")
df4 = pd.read_csv("data4.csv")

In [49]:
df = pd.concat([df0, df2, df4 ])

In [50]:
df = df.reset_index()

In [51]:
df

,index,Unnamed: 0.1,Unnamed: 0,url,content,useful
0,0,0,0,https://www.t3.com/reviews/secretlab-titan-evo...,The Secretlab Titan Evo Lite is a brand-new ch...,1
1,1,1,1,https://gamerant.com/secretlab-titan-evo-lite-...,"As Secretlab explains, the chair features 95% ...",1
2,2,2,2,https://www.gamespot.com/articles/secretlabs-n...,"Overall, Secretlab's Titan Evo Lite is a fanta...",1
3,3,3,3,https://www.hardwarezone.com.sg/review-secretl...,"Starting from S$519, Secretlab says this new v...",1
4,4,4,4,https://secretlab.eu/sv/products/titan-evo-202...,"Customer Reviews . 4.9 Based on 52,709 Reviews...",1
...,...,...,...,...,...,...
230,28,28,28,https://www.ign.com/articles/secretlab-titan-e...,No one likes the painful price most gaming cha...,1
231,29,29,29,https://secretlab.co/pages/titan-evo-2022-series,Gaming Chair Features | Secretlab TITAN Evo | ...,1
232,30,30,30,https://www.reddit.com/r/secretlab/comments/14...,A subreddit for all things Secretlab — we engi...,1
233,31,31,31,https://www.reddit.com/r/secretlab/comments/sh...,A subreddit for all things Secretlab — we engi...,1


In [52]:
df = df[["url", "content","useful"]]

In [53]:
df = df.drop_duplicates()

In [54]:
df

,url,content,useful
0,https://www.t3.com/reviews/secretlab-titan-evo...,The Secretlab Titan Evo Lite is a brand-new ch...,1
1,https://gamerant.com/secretlab-titan-evo-lite-...,"As Secretlab explains, the chair features 95% ...",1
2,https://www.gamespot.com/articles/secretlabs-n...,"Overall, Secretlab's Titan Evo Lite is a fanta...",1
3,https://www.hardwarezone.com.sg/review-secretl...,"Starting from S$519, Secretlab says this new v...",1
4,https://secretlab.eu/sv/products/titan-evo-202...,"Customer Reviews . 4.9 Based on 52,709 Reviews...",1
...,...,...,...
210,https://www.techradar.com/gaming/gaming-access...,The gaming furniture manufacturer Secretlab ha...,1
220,https://www.tomsguide.com/gaming/gaming-periph...,The Secretlab Titan Evo series is one of the b...,1
221,https://www.reddit.com/r/secretlab/comments/13...,Honest review: 2022 Titan EVO : r/secretlab Th...,1
225,https://www.pcgamer.com/hardware/shhh-dont-tel...,"Shhh, don't tell everyone, but the Secretlab T...",1


In [55]:
df['content'] = df['content'].str.split('.')
df = df.explode('content')

In [56]:
df= df.loc[df["content"].str.len()>= 2]

In [57]:
df = df.reset_index(drop=True)

In [58]:
import google.generativeai as genai
import os
import time

from dotenv import load_dotenv
client =  genai.configure(api_key=os.getenv("GOOGLE_GEMINI_API_KEY"))
# def gemini_usefulness(column):
#     new_list = []
#     for mehmeh in column:
#         model = genai.GenerativeModel('gemini-1.5-flash')
#         response = model.generate_content([
#     f"For the string provided, please respond with '1' if the data is relevant and important for a Secretlab Titan Evo Lite gaming chair product design review, and '0' if it is not : {mehmeh}"])
#         new_list.append(response.text)
#     return new_list
def gemini_usefulness(column):
    new_list = []
    batch_size = 5  # Process 5 rows at a time
    for i in range(0, len(column), batch_size):
        batch = column[i:i + batch_size]
        print(i)
        for mehmeh in batch:
            try:
                model = genai.GenerativeModel('gemini-2.0-flash')
                response = model.generate_content([
                    f"For the string provided, please respond with '1' if the data is relevant and important for a Secretlab Titan Evo Lite gaming chair product design review, and '0' if it is not : {mehmeh}"
                ])
                new_list.append(response.text)
            except Exception as e:
                print(f"An error occurred processing: {mehmeh}. Error: {e}")
                new_list.append(None) #Append None or some other error indicator
        if i + batch_size < len(column): # only sleep if there are more rows to process.
            print("Sleeping for half minute...")
            time.sleep(30)  # Sleep for 60 seconds (1 minute)
    return new_list      


c:\Users\Rald999\Documents\GitHub\product_research\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [59]:
def gemini_sentiment(column):
    new_list = []
    batch_size = 5  # Process 5 rows at a time
    for i in range(0, len(column), batch_size):
        batch = column[i:i + batch_size]
        print(i)
        for mehmeh in batch:
            try:
                model = genai.GenerativeModel('gemini-2.0-flash')
                response = model.generate_content([
                    f"Analyze the sentiment of the following text and return either positive, negative, or neutral: {mehmeh}"
                ])
                new_list.append(response.text)
            except Exception as e:
                print(f"An error occurred processing: {mehmeh}. Error: {e}")
                new_list.append(None) #Append None or some other error indicator
        if i + batch_size < len(column): # only sleep if there are more rows to process.
            print("Sleeping for 30 seconds...")
            time.sleep(30)  # Sleep for 60 seconds (1 minute)
    return new_list      


In [60]:
mehmehlist4 = gemini_usefulness(df["content"])

0
Sleeping for half minute...
5
Sleeping for half minute...
10
Sleeping for half minute...
15
Sleeping for half minute...
20
Sleeping for half minute...
25
Sleeping for half minute...
30
Sleeping for half minute...
35
Sleeping for half minute...
40
Sleeping for half minute...
45
Sleeping for half minute...
50
Sleeping for half minute...
55
Sleeping for half minute...
60
Sleeping for half minute...
65
Sleeping for half minute...
70
Sleeping for half minute...
75
Sleeping for half minute...
80
Sleeping for half minute...
85
Sleeping for half minute...
90
Sleeping for half minute...
95
Sleeping for half minute...
100
Sleeping for half minute...
105
Sleeping for half minute...
110
Sleeping for half minute...
115
Sleeping for half minute...
120
Sleeping for half minute...
125
Sleeping for half minute...
130
Sleeping for half minute...
135
Sleeping for half minute...
140
Sleeping for half minute...
145
Sleeping for half minute...
150
Sleeping for half minute...
155
Sleeping for half minute..

In [63]:
# mehmehlist4
df["usefulness2"] = mehmehlist4

In [65]:
df.to_csv("datacleaned.csv",index=False)

In [67]:
dfmehmeh = pd.read_csv("datacleaned.csv")
dfmehmeh


,url,content,useful,usefulness2
0,https://www.t3.com/reviews/secretlab-titan-evo...,The Secretlab Titan Evo Lite is a brand-new ch...,1,1\n
1,https://www.t3.com/reviews/secretlab-titan-evo...,"It is designed to offer ""95% of the premium e...",1,1\n
2,https://gamerant.com/secretlab-titan-evo-lite-...,"As Secretlab explains, the chair features 95% ...",1,1\n
3,https://gamerant.com/secretlab-titan-evo-lite-...,"And in our testing, the TITAN Evo Lite doesn'...",1,1\n
4,https://www.gamespot.com/articles/secretlabs-n...,"Overall, Secretlab's Titan Evo Lite is a fanta...",1,1\n
...,...,...,...,...
808,https://www.pcgamer.com/hardware/shhh-dont-tel...,Secretlab Titan Evo Lite | Leatherette or sof...,1,1\n
809,https://www.pcgamer.com/hardware/shhh-dont-tel...,Secretlab Titan Evo Lite | Leatherette or sof...,1,1\n
810,https://www.amazon.com/Secretlab-Titan-Black-L...,Secretlab Titan Evo Lite in Black Prime 2,1,1\n
811,https://www.amazon.com/Secretlab-Titan-Black-L...,0 Leatherette Gaming Chair - Reclining - Ergon...,1,1\n


In [70]:
dfmehmeh2 = dfmehmeh[dfmehmeh["usefulness2"]=="1\n"]
dfmehmeh2

,url,content,useful,usefulness2
0,https://www.t3.com/reviews/secretlab-titan-evo...,The Secretlab Titan Evo Lite is a brand-new ch...,1,1\n
1,https://www.t3.com/reviews/secretlab-titan-evo...,"It is designed to offer ""95% of the premium e...",1,1\n
2,https://gamerant.com/secretlab-titan-evo-lite-...,"As Secretlab explains, the chair features 95% ...",1,1\n
3,https://gamerant.com/secretlab-titan-evo-lite-...,"And in our testing, the TITAN Evo Lite doesn'...",1,1\n
4,https://www.gamespot.com/articles/secretlabs-n...,"Overall, Secretlab's Titan Evo Lite is a fanta...",1,1\n
...,...,...,...,...
808,https://www.pcgamer.com/hardware/shhh-dont-tel...,Secretlab Titan Evo Lite | Leatherette or sof...,1,1\n
809,https://www.pcgamer.com/hardware/shhh-dont-tel...,Secretlab Titan Evo Lite | Leatherette or sof...,1,1\n
810,https://www.amazon.com/Secretlab-Titan-Black-L...,Secretlab Titan Evo Lite in Black Prime 2,1,1\n
811,https://www.amazon.com/Secretlab-Titan-Black-L...,0 Leatherette Gaming Chair - Reclining - Ergon...,1,1\n


In [71]:
dfmehmeh2.to_csv("datacleaned2.csv",index=False)

In [ ]:
mehmehlist5 = gemini_sentiment(dfmehmeh2["content"])

0
Sleeping for 30 seconds...
5
Sleeping for 30 seconds...
10
Sleeping for 30 seconds...
15
Sleeping for 30 seconds...
20
Sleeping for 30 seconds...
25
Sleeping for 30 seconds...
30
Sleeping for 30 seconds...
35
Sleeping for 30 seconds...


In [ ]:
mehmehlist5

NameError: name 'mehmehlist5' is not defined

In [ ]:
df = df.reset_index()

In [ ]:
mehmehlist6 = gemini_usefulness(df["content"])

Sleeping for half minute...
Sleeping for half minute...
Sleeping for half minute...
Sleeping for half minute...
Sleeping for half minute...
Sleeping for half minute...
Sleeping for half minute...
Sleeping for half minute...
Sleeping for half minute...
Sleeping for half minute...
Sleeping for half minute...
Sleeping for half minute...
Sleeping for half minute...
Sleeping for half minute...
Sleeping for half minute...
Sleeping for half minute...
Sleeping for half minute...
Sleeping for half minute...
Sleeping for half minute...
Sleeping for half minute...
Sleeping for half minute...
Sleeping for half minute...
Sleeping for half minute...
Sleeping for half minute...
Sleeping for half minute...
Sleeping for half minute...
Sleeping for half minute...
Sleeping for half minute...
Sleeping for half minute...
Sleeping for half minute...
Sleeping for half minute...


Sleeping for half minute...
Sleeping for half minute...
Sleeping for half minute...
Sleeping for half minute...
Sleeping for half minute...
Sleeping for half minute...
Sleeping for half minute...
Sleeping for half minute...
Sleeping for half minute...
Sleeping for half minute...
Sleeping for half minute...
Sleeping for half minute...
Sleeping for half minute...
Sleeping for half minute...
Sleeping for half minute...
Sleeping for half minute...
Sleeping for half minute...
Sleeping for half minute...
Sleeping for half minute...
Sleeping for half minute...
Sleeping for half minute...
Sleeping for half minute...
Sleeping for half minute...
Sleeping for half minute...
Sleeping for half minute...
Sleeping for half minute...


In [ ]:
mehmehlist4

['1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '0\n',
 '1\n',
 '1\n',
 '0\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '0\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '0\n',
 '0\n',
 '1\n',
 '0\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '0\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',
 '1\n',


In [ ]:
df4["useful"] = mehmehlist3

In [ ]:
df2["useful"] = mehmehlist2

In [ ]:
tempdf["useful"] =mehmehlist4

In [ ]:
tempdf

,Unnamed: 0,url,content,useful
0,0,https://www.t3.com/reviews/secretlab-titan-evo...,The Secretlab Titan Evo Lite is a brand-new ch...,1\n
1,1,https://gamerant.com/secretlab-titan-evo-lite-...,"As Secretlab explains, the chair features 95% ...",1\n
2,2,https://www.gamespot.com/articles/secretlabs-n...,"Overall, Secretlab's Titan Evo Lite is a fanta...",1\n
3,3,https://www.hardwarezone.com.sg/review-secretl...,"Starting from S$519, Secretlab says this new v...",1\n
4,4,https://secretlab.eu/sv/products/titan-evo-202...,"Customer Reviews . 4.9 Based on 52,709 Reviews...",1\n
...,...,...,...,...
77,77,https://www.tiktok.com/@zachstechturf/video/74...,Gaming Chair Temu Review · Guy behind ... Secr...,1\n
78,78,https://m.youtube.com/watch?v=meDhajzejwM&pp=y...,Comments · What Size Titan Evo 2022 Should You...,1\n
79,79,https://www.tiktok.com/discover/secret-lab-tit...,I have not sit in this chair yet. I'm about to...,0\n
80,80,https://www.youtube.com/watch?v=FEGM3Lh2RKU&pp...,Support the channel and BUY IT HERE: Secretlab...,0\n


In [ ]:
tempdf.to_csv("data0.csv")

In [ ]:
df2.to_csv("data2.csv")
df4.to_csv("data4.csv")


In [ ]:
mehmehlist

['1\n', '1\n', '1\n', '1\n', '1\n']

In [ ]:
df4["content"].head(5)

0    The Secretlab Titan Evo Lite is a brand-new ch...
1    As Secretlab explains, the chair features 95% ...
2    In gaming chairs, Secretlab has left a mark as...
3    Starting from S$519, Secretlab says this new v...
4    The Secretlab Titan Evo 2022 is a firm gaming ...
Name: content, dtype: object

In [ ]:
pront(xxxxxxx)

NameError: name 'pront' is not defined

In [ ]:
import re

def clean_text(text):                               # user defined function for cleaning text
    text = text.lower()                             # all lower case
    text = re.sub(r'\[.*?\]', ' ', text)            # remove text within [ ] (' ' instead of '')
    text = re.sub(r'\<.*?\>', ' ', text)            # remove text within < > (' ' instead of '')
    text = re.sub(r'http\S+', ' ', text)            # remove website ref http
    text = re.sub(r'www\S+', ' ', text)             # remove website ref www

    text = text.replace('€', 'euros')               # replace special character with words
    text = text.replace('£', 'gbp')                 # replace special character with words
    text = text.replace('$', 'dollar')              # replace special character with words
    text = text.replace('%', 'percent')             # replace special character with words
    text = text.replace('\n', ' ')                  # remove \n in text that has it

    text = text.replace('\'', '’')                  # standardise apostrophe
    text = text.replace('&#39;', '’')               # standardise apostrophe

    text = text.replace('’d', ' would')             # remove ’ (for would, should? could? had + PP?)
    text = text.replace('’s', ' is')                # remove ’ (for is, John's + N?)
    text = text.replace('’re', ' are')              # remove ’ (for are)
    text = text.replace('’ll', ' will')             # remove ’ (for will)
    text = text.replace('’ve', ' have')             # remove ’ (for have)
    text = text.replace('’m', ' am')                # remove ’ (for am)
    text = text.replace('can’t', 'can not')         # remove ’ (for can't)
    text = text.replace('won’t', 'will not')        # remove ’ (for won't)
    text = text.replace('n’t', ' not')              # remove ’ (for don't, doesn't)

    text = text.replace('’', ' ')                   # remove apostrophe (in general)
    text = text.replace('&quot;', ' ')              # remove quotation sign (in general)

    text = text.replace('cant', 'can not')          # typo 'can't' (note that cant is a proper word)
    text = text.replace('dont', 'do not')           # typo 'don't'

    text = re.sub(r'[^a-zA-Z0-9]', r' ', text)      # only alphanumeric left
    text = text.replace("   ", ' ')                 # remove triple empty space
    text = text.replace("  ", ' ')                  # remove double empty space
    return text

# for i in range(len(comment_list)):
#     print(comment_list[i])                          # pre-clean
#     comment_list[i] = clean_text(comment_list[i])   # overwrite with clean_text function
#     print(comment_list[i])      

In [ ]:
df["content"] = df["content"].apply(clean_text)

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = stopwords.words('english')   

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Rald999\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def remove_stopword(text):
   
   text = " ".join(word for word in text.split() if word not in stopwords)
   return text


In [ ]:
df["content"] = df["content"].apply(remove_stopword)

In [ ]:
mehmehstring = ' '.join(df['content'].tolist())

In [ ]:
mehmehstring

'secretlab titan evo lite brand new chair one biggest names gaming chairs designed offer 95percent premium experience secretlab titan evo 20percent secretlab explains chair features 95percent titan evo features 20percent less cost gamers testing titan evo lite feel like downgrade overall secretlab titan evo lite fantastic gaming chair price point though original titan evo bit versatility lite offers many impactful features starting sdollar519 secretlab says new variant designed offer accessible price point without compromising quality comfort titan evo chairs known titan evo goes lite glance titan evo lite looks identical expensive predecessor customer reviews 4 9 based 52 709 reviews 5 secretlab titan evo titan evo lite chairs sale eligible 30 day free returns exception secretlab automobili lamborghini collection secretlab titan evo house dragon edition lacks standard evo adjustable lumbar support uses upholstery materials last seen 2020 titan line titan evo lite sturdy comfortable ch

In [ ]:
from collections import Counter
from nltk.util import everygrams

print("Finding top words...")

top_length = 3                # top_length > 1 not useful after removing stopwords
num_top = 30

e_grams_counts = Counter(everygrams(mehmehstring.split(), max_len=top_length))
e_grams_most = e_grams_counts.most_common(num_top)
print(e_grams_most)
print("Number of e-grams found:", len(e_grams_most))

Finding top words...
[(('secretlab',), 143), (('titan',), 127), (('evo',), 120), (('titan', 'evo'), 118), (('chair',), 95), (('gaming',), 93), (('series',), 87), (('reddit',), 83), (('secretlab', 'titan'), 78), (('movies',), 77), (('movies', 'series'), 77), (('secretlab', 'titan', 'evo'), 72), (('gaming', 'chair'), 48), (('lite',), 43), (('titan', 'evo', 'lite'), 38), (('evo', 'lite'), 38), (('chairs',), 28), (('reviews',), 28), (('r',), 27), (('new',), 26), (('posts',), 23), (('news',), 23), (('best',), 22), (('open',), 18), (('reddit', 'reddit'), 18), (('features',), 17), (('top',), 17), (('discussion',), 17), (('review',), 16), (('subreddit',), 16)]
Number of e-grams found: 30


: 